In [ ]:
import numpy as np
import torch

from PIL import Image
from matplotlib import pyplot as plt
from torchvision import transforms
from scipy.ndimage.filters import gaussian_filter

from datasets.data_util import *
from datasets.calc_scale import *

mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

trans = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])

def image_convert(img):
    _, c, _, _ = img.size()
    img = img.squeeze()
    if c > 3:
        img = img[0:3,:,:].to('cpu').detach().numpy().copy()
    else:
        img = img[:,:,:].to('cpu').detach().numpy().copy()

    img = img.transpose(1,2,0)
    img = img*std+mean

    return img

def density_convert(img):
    img = img.squeeze()
    img = img[:,:].to('cpu').detach().numpy().copy().squeeze()
    return img

def create_density(gt_map, sigma):
    gt_map = np.array(gt_map)
    density = gaussian_filter(gt_map, sigma)
    density = Image.fromarray(density)
    return density

In [ ]:
in_size = (1024, 683)
pool_num = 5
up_scale = 1

out_size = calc_out_size('resnet50', in_size, pool_num, up_scale)
scale_factor = calc_scale_factor(in_size, out_size)
print('output size: {}, scale factor: {}'.format(out_size, scale_factor))

out_size = calc_out_size('bagnet33', in_size, pool_num, up_scale)
scale_factor = calc_scale_factor(in_size, out_size)
print('output size: {}, scale factor: {}'.format(out_size, scale_factor))

out_size = calc_out_size('fusionnet', in_size, pool_num, up_scale)
scale_factor = calc_scale_factor(in_size, out_size)
print('output size: {}, scale factor: {}'.format(out_size, scale_factor))

In [ ]:
## GT: 400
image = Image.open('/mnt/hdd02/ShanghaiTech/part_A/test_data/images/IMG_179.jpg')
density = create_density(mapping_gt(image.size, load_gt(gt_path=('/mnt/hdd02/ShanghaiTech/part_A/test_data/ground_truth/GT_IMG_179.mat'))), sigma=15)

top = image.crop((0, 0, 1024, 342))
bottom = image.crop((0, 342, 1024, 683))

top_den = density.crop((0, 0, 1024, 342))
bottom_den = density.crop((0, 342, 1024, 683))

print(np.sum(top_den).sum(), np.sum(bottom_den).sum())

plt.imshow(np.asarray(top_den), cmap='jet')
plt.axis("off")

In [ ]:
## resnet50
from models.resnet import ResNet
resnet_partA_path = '/mnt/hdd02/res-bagnet/sta-resnet50-pretrained/best_model.pth' 

device = torch.device('cuda')
model = ResNet(arch='resnet50', pool_num=5, up_scale=1)

model.to(device)
model.load_state_dict(torch.load(resnet_partA_path))
model.eval()

image = trans(image)
image = image.unsqueeze(0).to(device)
density = model(image)

density = np.squeeze(density_convert(density))
print(density.sum())
plt.imshow(density, cmap='jet')
plt.axis("off")

In [ ]:
top_den = density[0:11,:]
bottom_den = density[11:,:]
print(top_den.sum(), bottom_den.sum())

In [ ]:
plt.imshow(top_den, cmap='jet')
plt.axis("off")

In [ ]:
plt.imshow(bottom_den, cmap='jet')
plt.axis("off")

In [ ]:
## bagnet33
from models.bagnet import BagNet
bagnet_partA_path = '/mnt/hdd02/res-bagnet/sta-bagnet33-pretrained/best_model.pth'

device = torch.device('cuda')
model = BagNet(arch='bagnet33', pool_num=5, up_scale=1)

model.to(device)
model.load_state_dict(torch.load(bagnet_partA_path))
model.eval()

image = trans(image)
image = image.unsqueeze(0).to(device)
density = model(image)

density = np.squeeze(density_convert(density))
print(density.sum())
plt.imshow(density, cmap='jet')
plt.axis("off")

In [ ]:
top_den = density[0:41,:]
bottom_den = density[41:,:]
print(top_den.sum(), bottom_den.sum())

In [ ]:
plt.imshow(top_den, cmap='jet')
plt.axis("off")

In [ ]:
plt.imshow(bottom_den, cmap='jet')
plt.axis("off")

In [ ]:
## fusion net
from models.fusion_model import BagResNet
fusion_partA_path = '/mnt/hdd02/res-bagnet/sta-fusionnet-200ep/200_ckpt.tar'

device = torch.device('cuda')
model = BagResNet(bag_arch='bagnet33', res_arch='resnet50', pool_num=5)

model.to(device)
model.load_state_dict(torch.load(fusion_partA_path)['model_state_dict'])
model.eval()

image = trans(image)
image = image.unsqueeze(0).to(device)
density = model(image)

density = np.squeeze(density_convert(density))
print(density.sum())
plt.imshow(density, cmap='jet')
plt.axis("off")

In [ ]:
top_den = density[0:26,:]
bottom_den = density[26:,:]
print(top_den.sum(), bottom_den.sum())

In [ ]:
plt.imshow(top_den, cmap='jet')
plt.axis("off")

In [ ]:
plt.imshow(bottom_den, cmap='jet')
plt.axis("off")